## Attention Model

### 양방향 LSTM과 어텐션 매커니즘으로 IMDB 리뷰 감성 분류

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences 

In [3]:
# 최대 단어 개수를 만개로 제한하고 훈련 데이터와 테스트 데이터 받아옴 

vocab_size = 10000
(X_train, y_train) , (X_test , y_test) = imdb.load_data(num_words = vocab_size)

17473536/17464789 [==============================] - 3s 0us/step


- 훈련 데이터와 이에 대한 레이블이 각각 X_train, y_train에 테스트 데이터와 이에 대한 레이블이 각각 X_test, y_test에 저장됨.
- IMDB 리뷰 데이터는 이미 정수 인코딩이 된 상태므로 남은 전처리는 패딩뿐임
- 리뷰의 최대 길이와 평균 길이 확인

In [6]:
X_train[:1]

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])],
      dtype=object)

In [7]:
print('리뷰의 최대 길이 : {}'.format(max(len(l) for l in X_train)))
print('리뷰의 평균 길이 : {}'.format(sum(map(len, X_train))/len(X_train)))

리뷰의 최대 길이 : 2494
리뷰의 평균 길이 : 238.71364


* 평균 길이보다는 조금 크게 데이터 패딩 

In [10]:
# 병렬 연산을 위해서 여러 문장의 길이를 임의로 동일하게 맞춰주는 작업진행 
max_len =500
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test , maxlen= max_len)

In [13]:
print('리뷰의 최대 길이 : {}'.format(max(len(l) for l in X_train)))
print('리뷰의 평균 길이 : {}'.format(sum(map(len, X_train))/len(X_train)))

리뷰의 최대 길이 : 500
리뷰의 평균 길이 : 500.0


### 바다나우 어텐션(Bahdanau Attention)

- 어텐션 스코어 함수: 주어진 query와 모든 key에 대해서 유사도를 측정하는 함수
- 닷 프로덕트 어텐션에서는 query와 key의 유사도를 구하는 방법이 내적(dot product)
  - 닷 프로덕트 어텐션의 어텐션 스코어 함수


- 바다나우 어텐션의 어텐션 스코어 함수


- 텍스트 분류에서 어텐션 메커니즘을 사용하는 이유
  - RNN의 마지막 은닉 상태는 예측을 위해 사용
  - 그런데 이 RNN의 마지막 은닉 상태는 몇 가지 유용한 정보들을 손실한 상태임
  - 그래서 RNN이 time step을 지나며 손실했던 정보들을 다시 참고하고자 함
  - 이는 다시 말해 RNN의 모든 은닉 상태들을 다시 한 번 참고하겠다는 것
  - 이를 위해서 어텐션메커니즘을 사용함

In [14]:
import tensorflow as tf

In [15]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)


  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

### 양방향 LSTM + 어텐션 메커니즘(BiLSTM with Attention Mechanism)

In [16]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

### 모델 설계

- 10,000개의 단어들을 128차원의 벡터로 임베딩하도록 설계
- 양방향 LSTM 설계
  - 여기서는 양방향 LSTM을 두 층을 사용
  - 첫번째 층
    - 두번째 층을 위에 쌓을 예정이므로 return_sequences를 True로 설정
  - 두번째 층
    - 상태를 리턴받아야 하므로 return_state를 True로 설정

In [17]:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero = True)(sequence_input)

In [18]:
lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences = True))(embedded_sequences)

In [19]:
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

In [20]:
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


- 순방향 LSTM의 은닉 상태와 셀상태를 forward_h, forward_c에 저장
- 역방향 LSTM의 은닉 상태와 셀 상태를 backward_h, backward_c에 저장
<br><br>
- 각 은닉 상태나 셀 상태의 경우에는 128차원을 가지는데, lstm의 경우에는 (500 × 128)의 크기를 가짐. foward 방향과 backward 방향이 연결된 hidden state벡터가 모든 시점에 대해서 존재함을 의미함

- 양방향 LSTM을 사용할 경우에는 순방향 LSTM과 역방향 LSTM 각각 은닉 상태와 셀 상태를 가지므로, 양방향 LSTM의 은닉 상태와 셀 상태를 사용하려면 두 방향의 LSTM의 상태들을 연결(concatenate)함

In [21]:
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

- 은닉 상태를 입력으로 하여 컨텍스트 벡터(context vector) 획득

In [22]:
attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

- 컨텍스트 벡터를 밀집층(dense layer)에 통과시키고 이진 분류이므로 최종 출력층에 1개의 뉴런을 배치하고 활성화 함수로 시그모이드 함수를 사용

In [23]:
dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)

- 옵티마이저로 아담 옵티마이저 사용하고, 모델을 컴파일
- 시그모이드 함수를 사용하므로 손실 함수로 binary_crossentropy를 사용

In [24]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### 모델 학습

- 검증 데이터로 테스트 데이터를 사용하여 에포크가 끝날 때마다 테스트 데이터에 대한 정확도를 출력

In [ ]:
history = model.fit(X_train, y_train, epochs = 3, batch_size = 256, validation_data=(X_test, y_test), verbose=1)

Epoch 1/3
 1/98 [..............................] - ETA: 37:10:02 - loss: 0.6931 - accuracy: 0.5391

In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))